# Get data and packages

In [3]:
%%capture
!aws s3 cp --recursive s3://xdss-public-datasets/demos/taxi_parquet ../datasets/taxi_parquet

In [1]:
import gc
from src.benchmarks_utils import benchmark, get_results
from src.spark_utils import *
from src.config import repetitions


name = 'spark'
data_path = '../datasets/taxi_parquet/'
results_path = f"../results/{name}_1b_mlm52xlarge.csv"
benchmarks = {}

# Benchmarks

In [2]:
# Load data
data = read_file(data_path=data_path)
data = data.withColumn('pickup_hour', sql.functions.split(data['pickup_datetime'], ' ').getItem(1))
print(f"size: {data.count()} with {len(data.columns)} columns")

size: 1173057928 with 19 columns


In [9]:
benchmarks['read_file']= benchmark(read_file, df=data, data_path=data_path, repetitions=repetitions)
benchmarks['mean']= benchmark(mean, data, repetitions=repetitions)
benchmarks['standard deviation']= benchmark(standard_deviation, data, repetitions=repetitions)
benchmarks['sum columns']= benchmark(sum_columns, data, repetitions=repetitions)
benchmarks['product columns']= benchmark(product_columns, data, repetitions=repetitions)
benchmarks['arithmetic operation']= benchmark(complicated_arithmetic_operation, data, repetitions=repetitions)
benchmarks['value counts']= benchmark(value_counts, data, repetitions=repetitions)
benchmarks['groupby statistics']= benchmark(groupby_statistics, data, repetitions=repetitions)
benchmarks['filter']= benchmark(filter_data, data, repetitions=repetitions)

In [10]:
# filtered
filterd = filter_data(data)
del data

print(f"Prepare filtered data and deleted {gc.collect()} MB")
benchmarks['filtered mean'] = benchmark(mean, filterd, repetitions=repetitions)
benchmarks['filtered standard deviation'] = benchmark(standard_deviation, filterd, repetitions=repetitions)
benchmarks['filtered sum columns'] = benchmark(sum_columns , filterd, repetitions=repetitions)
benchmarks['filtered product_columns'] = benchmark(product_columns , filterd, repetitions=repetitions)
benchmarks['filtered complicated arithmetic operation'] = benchmark(complicated_arithmetic_operation, filterd, repetitions=repetitions)
benchmarks['filtered value counts'] = benchmark(value_counts, filterd, repetitions=repetitions)
benchmarks['filtered groupby statistics'] = benchmark(groupby_statistics, filterd, repetitions=repetitions)

print(f"Done benchmarks on filterd data")

Prepare filtered data and deleted 38 MB
Done benchmarks on filterd data


In [11]:
print(f"cleaned {gc.collect()} mb")
get_results(benchmarks, name).to_csv(results_path)
!aws s3 cp  ../results/spark_1b_mlm52xlarge.csv s3://vaex-sagemaker-demo/benchmarks/spark_1b_mlm52xlarge_results.csv 

cleaned 97 mb
upload: ../results/spark_1b_mlm52xlarge.csv to s3://vaex-sagemaker-demo/benchmarks/spark_1b_mlm52xlarge_results.csv


In [12]:
benchmarks['filtered join'] = benchmark(join, filterd, repetitions=repetitions, other=groupby_statistics(filterd))

Py4JJavaError: An error occurred while calling o262.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 113 in stage 49.0 failed 1 times, most recent failure: Lost task 113.0 in stage 49.0 (TID 5043, localhost, executor driver): java.io.IOException: No space left on device
	at sun.nio.ch.FileDispatcherImpl.write0(Native Method)
	at sun.nio.ch.FileDispatcherImpl.write(FileDispatcherImpl.java:60)
	at sun.nio.ch.IOUtil.writeFromNativeBuffer(IOUtil.java:93)
	at sun.nio.ch.IOUtil.write(IOUtil.java:51)
	at sun.nio.ch.FileChannelImpl.write(FileChannelImpl.java:211)
	at sun.nio.ch.FileChannelImpl.transferToTrustedChannel(FileChannelImpl.java:516)
	at sun.nio.ch.FileChannelImpl.transferTo(FileChannelImpl.java:612)
	at org.apache.spark.util.Utils$.copyFileStreamNIO(Utils.scala:377)
	at org.apache.spark.util.Utils$$anonfun$copyStream$1.apply$mcJ$sp(Utils.scala:342)
	at org.apache.spark.util.Utils$$anonfun$copyStream$1.apply(Utils.scala:336)
	at org.apache.sp

In [ ]:
results = get_results(benchmarks, name)
results.to_csv(results_path)
results.head()

In [ ]:
!aws s3 cp  ../results/spark_1b_mlm52xlarge.csv s3://vaex-sagemaker-demo/benchmarks/spark_1b_mlm52xlarge_results.csv 
    